- Note: Call per second is limited so need to save locally.
- File is too big, cannot save as 1 file
- Looking into removing parts that don't need to json file
    - Decide to use multithreading to download several at the same time
    
- need to explain gender rate (refer to doc)  
- Problem: Need to find a differrent source for sprite

https://pokeapi.co/docs/v2.html

## In order to reduce the number of call of API, we decided to save this url to local.
### Fixed limit: 100 API requests per IP address per minute

In [3]:
import requests
import json
import collections
import time
import concurrent.futures
import csv
import functools
from pprint import pprint
from typing import List, Dict
from IPython.display import clear_output

Each pokemon has many different json files with different information. This will run through each pokemon in each json type (input) and save them as a csv file. This is how it looks like at the end:

```
bulbasaur,https://pokeapi.co/api/v2/pokemon-species/1/
ivysaur,https://pokeapi.co/api/v2/pokemon-species/2/
venusaur,https://pokeapi.co/api/v2/pokemon-species/3/
charmander,https://pokeapi.co/api/v2/pokemon-species/4/
charmeleon,https://pokeapi.co/api/v2/pokemon-species/5/
charizard,https://pokeapi.co/api/v2/pokemon-species/6/
squirtle,https://pokeapi.co/api/v2/pokemon-species/7/
wartortle,https://pokeapi.co/api/v2/pokemon-species/8/
blastoise,https://pokeapi.co/api/v2/pokemon-species/9/
caterpie,https://pokeapi.co/api/v2/pokemon-species/10/
...
```


In [1]:
def request_and_save_urls(json_type: str):
    # Each url only has 20 pokemons
    url = fr"https://pokeapi.co/api/v2/{json_type}/"
    with open(f"data/{json_type}/{json_type}-urls.csv", "w") as file:
        pokemon_url_writer = csv.writer(file, delimiter=",", lineterminator="\n")

        while 1:
            # Get urls of 20 pokemons
            response = requests.get(url)
            json_data = json.loads(response.text)

            # Write to CSV file
            for pokemon in json_data["results"]:
                pokemon_url_writer.writerow(list(pokemon.values()))

            # Next 20 pokemons
            url = json_data["next"]
            if not url: break

### We want the following json files

In [4]:
request_and_save_urls("pokemon")
request_and_save_urls("pokemon-species")

### Load the csv file and request each Pokemon
This is used to fetch data from pokeAPI and save it locally. We load the CSV file earlier and request each Pokemon information. We group all information by type (pokemon_species, ...) and save it as a JSON file. However, because of the **fixed limit** on how many API can be called per minute (100), there is a delay in the code which make the total run time between 15-20 minutes

We also have an exclusion list to exclude data that are not needed from the responsed JSON

In [3]:
def request_and_cache(json_type, exclude: List[str]) -> None:
    return_json = {}
    
    # Open csv file to get urls
    pokemon_urls = list(csv.reader(open(f"data/{json_type}/{json_type}-urls.csv", "r"), delimiter=","))
    
    # Request each url and save as 1 big json file
    for i, (pokemon_name, pokemon_url) in enumerate(pokemon_urls):
        clear_output()
        print(f"{i}. Working on type: '{json_type}', pokemon: {pokemon_name}")
        pokemon_json = json.loads(requests.get(pokemon_url).text)
        
        # To reduce the file size, we exclude some parts of json_data
        return_json[pokemon_name] = {k:v for k, v in pokemon_json.items() if k not in exclude}
        
        # Make sure not to exceed to API limit
        time.sleep(1)
    json.dump(return_json, open(f"data/{json_type}/{json_type}.json", "w"))

We use multithreading to make this process faster

In [4]:
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for json_type, exclude in [("pokemon", ["game_indices", 
                                            "moves", 
                                            ]), 
                               ("pokemon-species", ["genera", 
                                                    "names", 
                                                    "pokedex_numbers", 
                                                    "varieties", 
                                                    "flavor_text_entries",
                                                   ]),
                               ]:
        temp = functools.partial(request_and_cache, exclude=exclude)
        executor.map(temp, [json_type])

963. Working on type: 'pokemon', pokemon: necrozma-ultra


### Load Pokemon Json files and Analyse

#### Pokemon

In [5]:
all_pokemons_info = collections.defaultdict(dict)

pokemon_json = json.load(open("data/pokemon/pokemon.json", "r"))
for pokemon, info in pokemon_json.items():
    pokemon_info = all_pokemons_info[pokemon]
    
    # Get height, weight, name
    for key in ("height", "weight", "name"):
        pokemon_info[key] = info[key]
        
    # Get 6 stats (hp, att, def, sp. att, sp. def) for pokemon
    for stats in info["stats"]:
        pokemon_info[stats["stat"]["name"]] = stats["base_stat"]
    
    # Get list of abilities
    pokemon_info["abilities"] = [ability["ability"]["name"] for ability in info["abilities"]]
    
    # Get Pokemon Image
    pokemon_info["sprites"] = info["sprites"]["front_default"]
    
    # Get Pokemon Types
    pokemon_info["types"] = [type_["type"]["name"] for type_ in info["types"]]
    
    # Get Pokemon Species Name
    pokemon_info["species"] = info["species"]["name"]

#### Pokemon Species

In [6]:
pokemon_species_json = json.load(open("data/pokemon-species/pokemon-species.json", "r"))
for pokemon, info in pokemon_species_json.items():
    pokemon_info = all_pokemons_info[pokemon]
    
    # Get basic info
    for key in ("base_happiness", "capture_rate", "gender_rate"):
        pokemon_info[key] = info[key]
    
    # Get list of egg groups
    pokemon_info["egg_groups"] = [group["name"] for group in info["egg_groups"]]
    
    # Get Evolution Chain
    if info["evolves_from_species"]:
        pokemon_info["evolves_from"] = info["evolves_from_species"]["name"]
        prev_pokemon = all_pokemons_info[pokemon_info["evolves_from"]]
        prev_pokemon["evolve_to"] = prev_pokemon.get("evolve_to", []) + [pokemon]    
        
    # Get basic info. These info are in a dictionary
    for key in ("color", "generation", "growth_rate", "habitat", "shape"):
        if info[key]: pokemon_info[key] = info[key]["name"]    

In [7]:
json.dump(all_pokemons_info, open("data/all_pokemons_info.json", "w"))

### Convert from JSON to CSV

In [8]:
all_pokemons_info_csv = []

with open("data/all_pokemons_info.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter=",", lineterminator="\n")
    for pokemon, info in all_pokemons_info.items():
        try:
            writer.writerow([info["sprites"],
                             pokemon,
                             ", ".join(info["types"]),
                             info["hp"],
                             info["attack"],
                             info["defense"],
                             info["special-attack"],
                             info["special-defense"],
                             info["speed"],
                             ", ".join(info["abilities"]),
    #                         info["base_happiness"],
    #                         info["capture_rate"],
                             info["height"]*10,
                             info["weight"]/10,])
        except:
            print(pokemon, end=", ")
    
        

deoxys, wormadam, giratina, shaymin, basculin, darmanitan, tornadus, thundurus, landorus, keldeo, meloetta, meowstic, aegislash, pumpkaboo, gourgeist, oricorio, lycanroc, wishiwashi, minior, mimikyu, 

# Print using CSV

In [9]:
from IPython.display import HTML, display
import tabulate
import pandas as pd

content = [["#", "Sprite", "Name", "Types", 
            "HP", "Att", "Def", "Sp. Att", "Sp. Def", "Speed", 
            "Abilities", "Base Happiness", "Capture Rate",
            "Height (cm)", "Weight (kg)"]]
with open("data/all_pokemons_info.csv", "r") as csv_file:
    csv_data = csv.reader(csv_file)
    for index, info in enumerate(csv_data):
        sprites, pokemon, types, hp, attack, defense, special_attack, special_defense, speed, abilities, height, weight = info
        content.append([index + 1,
                        f"<img src='{sprites}' alt='{pokemon}'",
                        pokemon,
                        types,
                        hp,
                        attack,
                        defense,
                        special_attack,
                        special_defense,
                        speed,
                        abilities,
                        height,
                        weight,
                       ])
display(HTML(tabulate.tabulate(content[:50], 
                               headers='firstrow', 
                               tablefmt='html',
                               
                              )))

#,Sprite,Name,Types,HP,Att,Def,Sp. Att,Sp. Def,Speed,Abilities,Base Happiness,Capture Rate
1,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png' alt='bulbasaur',bulbasaur,"poison, grass",45,49,49,65,65,45,"chlorophyll, overgrow",70,6.9
2,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/2.png' alt='ivysaur',ivysaur,"poison, grass",60,62,63,80,80,60,"chlorophyll, overgrow",100,13
3,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/3.png' alt='venusaur',venusaur,"poison, grass",80,82,83,100,100,80,"chlorophyll, overgrow",200,100
4,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/4.png' alt='charmander',charmander,fire,39,52,43,60,50,65,"solar-power, blaze",60,8.5
5,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/5.png' alt='charmeleon',charmeleon,fire,58,64,58,80,65,80,"solar-power, blaze",110,19
6,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/6.png' alt='charizard',charizard,"flying, fire",78,84,78,109,85,100,"solar-power, blaze",170,90.5
7,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/7.png' alt='squirtle',squirtle,water,44,48,65,50,64,43,"rain-dish, torrent",50,9
8,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/8.png' alt='wartortle',wartortle,water,59,63,80,65,80,58,"rain-dish, torrent",100,22.5
9,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/9.png' alt='blastoise',blastoise,water,79,83,100,85,105,78,"rain-dish, torrent",160,85.5
10,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/10.png' alt='caterpie',caterpie,bug,45,30,35,20,20,45,"run-away, shield-dust",30,2.9


# Print using JSON

In [10]:
from IPython.display import HTML, display
import tabulate

all_pokemons_info = json.load(open("data/all_pokemons_info.json", "r"))

content = [["#", "Sprite", "Name", "Types", 
            "HP", "Att", "Def", "Sp. Att", "Sp. Def", "Speed", 
            "Abilities", "Base Happiness", "Capture Rate",
            "Height (cm)", "Weight (kg)"]]
for i, (pokemon, info) in enumerate(all_pokemons_info.items()):
    try:
        content.append([i+1,
                        f"<img src='{info['sprites']}' alt='{pokemon}'",
                        pokemon,
                        "<br>".join(info["types"]),
                        info["hp"],
                        info["attack"],
                        info["defense"],
                        info["special-attack"],
                        info["special-defense"],
                        info["speed"],
                        "<br>".join(info["abilities"]),
                        info["height"]*10,
                        info["weight"]/10,
                     ])
    except:
        print(pokemon, end=", ")

display(HTML(tabulate.tabulate(content[:5], 
                               headers='firstrow', 
                               tablefmt='html',
                               
                              )))

deoxys, wormadam, giratina, shaymin, basculin, darmanitan, tornadus, thundurus, landorus, keldeo, meloetta, meowstic, aegislash, pumpkaboo, gourgeist, oricorio, lycanroc, wishiwashi, minior, mimikyu, 

#,Sprite,Name,Types,HP,Att,Def,Sp. Att,Sp. Def,Speed,Abilities,Base Happiness,Capture Rate
1,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/1.png' alt='bulbasaur',bulbasaur,poisongrass,45,49,49,65,65,45,chlorophyllovergrow,70,6.9
2,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/2.png' alt='ivysaur',ivysaur,poisongrass,60,62,63,80,80,60,chlorophyllovergrow,100,13
3,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/3.png' alt='venusaur',venusaur,poisongrass,80,82,83,100,100,80,chlorophyllovergrow,200,100
4,<img src='https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/4.png' alt='charmander',charmander,fire,39,52,43,60,50,65,solar-powerblaze,60,8.5
